# Phenolopy Metrics module tests

## Globals

In [ ]:
# set globals paths
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [ ]:
# imports
import os
import random
import numpy as np
import xarray as xr
from IPython.utils import io

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import full arcpy toolbox
arcpy.ImportToolbox(r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

### Reload libraries

In [ ]:
# if scripts change, reload
from importlib import reload
reload(test_funcs)

### Set data files and locations

In [ ]:
# setup general io. nc ins and outs exist in these folders
input_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs'
output_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\outputs'

# temp nc file for use when breaking ncs
temp_nc = os.path.join(input_folder, 'temp_nc.nc')  

# setup landsat cubes paths
ls_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_1_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc", 
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_ls_90_20_raw_odc.nc", 
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\dwer_1_ls_90_20_raw_odc.nc",  
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_1_ls_10_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_3_ls_10_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\tute_1_ls_10_20_raw_odc.nc",  
]

# setup sentinel2 cubes paths
s2_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_s2_16_20_raw_odc.nc", 
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\dwer_1_s2_16_20_raw_odc.nc",  
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_2_s2_16_21_raw_odc.nc",   
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\tute_1_s2_18_20_raw_odc.nc",
]

### Set specific raw netcdf file

In [ ]:
# set specific dataset
nc_file = ls_cubes[1]
#nc_file = s2_cubes[4]

### Set up function to iterate corruptor and tests

In [ ]:
def run_corruptors_through_tests(in_nc, nc_coors, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed.
    """
    
    for nc_corr in nc_corrs:
        corr_name = nc_corr[0].__name__                   # name of current corruptor func
        corr_func, corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params
        
        # notify
        print('Corrupting NetCDF via: {}.\n'.format(corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            corr_func(**corr_params)
        except Exception as e:    
            print(e)
            print('Corruptor did not have enough data to work with. Skipping.\n')

        # iter each test func and apply to current corrupt nc
        for test in tests:
            test_nc_name = corr_name + '_' + test[0]    # name of current test nc
            test_func, test_params = test[1], test[2]   # pointer to test func and dict of params
            test_msg = test[3]
            
            # notify of test message
            print(test_msg)

            # create output nc file path and name and update params for in/out paths
            out_nc_file = os.path.join(output_folder, test_nc_name)
            
            # remove output nc if exists
            if os.path.exists(out_nc_file):
                os.remove(out_nc_file)
            
            # update params
            test_params.update({'in_nc_file': temp_nc, 'out_likelihood_nc_file': out_nc_file})

            # perform current test
            try:
                # notify
                print('Performing test: {}.'.format(test_nc_name))

                # perform test, provide prints if requested
                if not verbose:
                    with io.capture_output() as cap:
                        test_func(**test_params)
                else:
                    test_func(**test_params)
                    print('\n')

            except Exception as e:    
                print(e)

        # notify
        print('All tests applied to corruptor NetCDF.\n\n')

### Set up netcdf corruptor functions

In [ ]:
# these are numerous netcdf corruptors. feed a raw nc in, break it, output as temp nc
# comment out any that are irrelevant
# each of these uncommented will be fed through the tests below
def build_nc_corruptors(temp_nc):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    cs = []

    # func: raw default dataset, no changes
    cs.append([test_funcs.nc_default, {'in_nc': temp_nc}])

    # func: remove x, y, time, spatial_ref coords
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'x'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'y'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'time'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'spatial_ref'}])

    #func: remove red and oa_fmask band vars
    #cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'nbart_red'}])
    #cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'oa_fmask'}])

    #func: limit number of years in various combos
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1990, 'e_year': 1990}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2010, 'e_year': 2010}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2012, 'e_year': 2012}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1991, 'e_year': 1992}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2005, 'e_year': 2006}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2019, 'e_year': 2020}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1993, 'e_year': 1995}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2010, 'e_year': 2012}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2011, 'e_year': 2013}])

    #func: set all vars to nan
    #cs.append([test_funcs.set_nc_vars_all_nan, {'in_nc': temp_nc}])

    #func: set all vars to zero
    #cs.append([test_funcs.set_nc_vars_all_zero, {'in_nc': temp_nc}])

    #func: set all vars for 10 rand times to all nan
    #cs.append([test_funcs.set_nc_vars_random_all_nan, {'in_nc': temp_nc, 'num': 10}])

    #func: strip all attrs from nc    
    #cs.append([test_funcs.strip_nc_attributes, {'in_nc': temp_nc}])

    #func: set vars in first and last time index to all nan
    #cs.append([test_funcs.set_end_times_to_all_nan, {'in_nc': temp_nc}])

    #func: reduce whole nc to one random time slice
    #cs.append([test_funcs.reduce_to_one_scene, {'in_nc': temp_nc}])

    #func: set wet months all nan, all years, for specific months
    #cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1]}])
    #cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1, 2, 3]}])
    #cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]}])

    #func: set wet months all nan, specific years, for specific months
    #cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [1990], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2005], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2006, 2007], 'months': [1, 2, 3]}])

    #func: drop wet months, all years, for specific months
    #cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1]}])
    #cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1, 2, 3]}])
    #cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]}])

    #func: drop wet months, specific years, for specific months (note: seperate tests)
    #cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [1990], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2009], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2007, 2008], 'months': [1, 2, 3]}])

    #func: remove crs attribute
    #cs.append([test_funcs.remove_crs_attr, {'in_nc': temp_nc}])

    #func: invalidate crs attribute
    #cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': 'EPSG:4326'}])
    #cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': ''}])

    #func: remove nodatavals attribute
    #cs.append([test_funcs.remove_nodatavals_attr, {'in_nc': temp_nc}])
    
    return cs

nc_corruptors = build_nc_corruptors(temp_nc=temp_nc)

## Run tests

### Test One: Wet Months

In [ ]:
def build_test_one_funcs(in_nc, temp_nc):
    """sets up test one functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '',                      # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []

    # func: default wet months 1, 2, 3
    msg = 'Running Test One: default wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': '1;2;3'})
    ts.append(['t_1_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: no wet months
    msg = 'Running Test One: "" wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': ''})
    ts.append(['t_1_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: one random month
    rand = str(random.randint(1, 5))
    msg = 'Running Test One: single random wet month ({}).'.format(rand)
    params = inputs.copy()
    params.update({'in_wet_months': rand})
    ts.append(['t_1_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: three months (2, 3, 4)
    msg = 'Running Test One: 2;3;4 as wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': '2;3;4'})
    ts.append(['t_1_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
    
    # func: all dry season months in wet season
    msg = 'Running Test One: dry months 5;6;7;8;9 as wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': '5;6;7;8;9'})
    ts.append(['t_1_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: wet season includes a month in dec
    msg = 'Running Test One: wet months contain dec (12).'
    params = inputs.copy()
    params.update({'in_wet_months': '12;1;2'})
    ts.append(['t_1_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: wet season includes only one month in dec
    msg = 'Running Test One: single wet month in dec (12).'
    params = inputs.copy()
    params.update({'in_wet_months': '12'})
    ts.append(['t_1_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     
        
    return ts

### Test One: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_one_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Two: Dry Months

In [ ]:
def build_test_two_funcs(in_nc, temp_nc):
    """sets up test two functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                      # wet months 
        'in_dry_months': '',                      # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default dry months 9, 10, 11
    msg = 'Running Test Two: default dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': '9;10;11'})
    ts.append(['t_2_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: no dry months
    msg = 'Running Test Two: "" dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': ''})
    ts.append(['t_2_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: one random month
    rand = str(random.randint(7, 12))
    msg = 'Running Test Two: single random dry month ({}).'.format(rand)
    params = inputs.copy()
    params.update({'in_dry_months': rand})
    ts.append(['t_2_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: three months (8, 9, 10)
    msg = 'Running Test Two: 10;11;12 as dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': '10;11;12'})
    ts.append(['t_2_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
    
    # func: all dry season months in wet season
    msg = 'Running Test Two: dry months 1;2;3;4;5 as dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': '1;2;3;4;5'})
    ts.append(['t_2_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: wet season includes a month in dec
    msg = 'Running Test Two: dry months contain one month in jan (1).'
    params = inputs.copy()
    params.update({'in_dry_months': '11;12;1'})
    ts.append(['t_2_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
        
    return ts

### Test Two: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_two_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Three: Vegetation Idx

In [ ]:
def build_test_three_funcs(in_nc, temp_nc):
    """sets up test three functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': '',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default veg idx mavi
    msg = 'Running Test Three: default veg idx (mavi).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MAVI'})
    ts.append(['t_3_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is ''
    msg = 'Running Test Three: veg idx is "".'
    params = inputs.copy()
    params.update({'in_veg_idx': ''})
    ts.append(['t_3_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is ndvi
    msg = 'Running Test Three: veg idx is NDVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NDVI'})
    ts.append(['t_3_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is evi
    msg = 'Running Test Three: veg idx is EVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'EVI'})
    ts.append(['t_3_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is savi
    msg = 'Running Test Three: veg idx is SAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'SAVI'})
    ts.append(['t_3_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is msavi
    msg = 'Running Test Three: veg idx is MSAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MSAVI'})
    ts.append(['t_3_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: veg idx is slavi
    msg = 'Running Test Three: veg idx is SLAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'SLAVI'})
    ts.append(['t_3_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: veg idx is mavi
    msg = 'Running Test Three: veg idx is MAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MAVI'})
    ts.append(['t_3_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is kndvi
    msg = 'Running Test Three: veg idx is kNDVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'kNDVI'})
    ts.append(['t_3_h.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: veg idx is tcg
    msg = 'Running Test Three: veg idx is TCG.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'TCG'})
    ts.append(['t_3_i.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
    
    # func: veg idx is nrva (non-existant)
    msg = 'Running Test Three: veg idx is NRVA (non-existant).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NRVA'})
    ts.append(['t_3_j.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
        
    return ts

### Test Three: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_three_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Four: Moisture Idx

In [ ]:
def build_test_four_funcs(in_nc, temp_nc):
    """sets up test four functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': '',                         # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default mst idx ndmi
    msg = 'Running Test Four: default mst idx (ndmi).'
    params = inputs.copy()
    params.update({'in_mst_idx': 'NDMI'})
    ts.append(['t_4_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: mst idx is ''
    msg = 'Running Test Four: mst idx is "".'
    params = inputs.copy()
    params.update({'in_mst_idx': ''})
    ts.append(['t_4_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: mst idx is gvmi
    msg = 'Running Test Four: mst idx is gvmi.'
    params = inputs.copy()
    params.update({'in_mst_idx': 'GVMI'})
    ts.append(['t_4_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: mst idx is rasd
    msg = 'Running Test Four: mst idx is rasd.'
    params = inputs.copy()
    params.update({'in_mst_idx': 'RASD'})
    ts.append(['t_4_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])   
            
    return ts

### Test Four: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_four_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Five: Aggregate

In [ ]:
def build_test_five_funcs(in_nc, temp_nc):
    """sets up test five functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': '',                         # moisture index name       
        'in_aggregate': True,                     # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default aggregate (True)
    msg = 'Running Test Five: default aggregate (True).'
    params = inputs.copy()
    params.update({'in_aggregate': True})
    ts.append(['t_5_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: aggregate is False
    msg = 'Running Test Five: aggregate is False'
    params = inputs.copy()
    params.update({'in_aggregate': False})
    ts.append(['t_5_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: aggregate is None
    msg = 'Running Test Five: aggregate is None'
    params = inputs.copy()
    params.update({'in_aggregate': None})
    ts.append(['t_5_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     
            
    return ts

### Test Five: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_five_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Six: Outlier Correction

In [ ]:
def build_test_six_funcs(in_nc, temp_nc):
    """sets up test six functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default zscore p-value
    msg = 'Running Test Five: default zscore p-value (None).'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': None})
    ts.append(['t_6_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: zscore p-value is ''
    msg = 'Running Test Five: zscore p-value is "".'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': ''})
    ts.append(['t_6_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
 
    # func: zscore p-value is string of 0.01
    msg = 'Running Test Five: zscore p-value is 0.01.'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': 0.01})
    ts.append(['t_6_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: zscore p-value is string of 0.05
    msg = 'Running Test Five: zscore p-value is 0.05.'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': 0.05})
    ts.append(['t_6_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: zscore p-value is string of 0.1
    msg = 'Running Test Five: zscore p-value is 0.1.'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': 0.1})
    ts.append(['t_6_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: zscore p-value is string of 1.0 (not supported)
    msg = 'Running Test Five: zscore p-value is 1.0 (not supported).'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': 1.0})
    ts.append(['t_6_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: zscore p-value is float of 0.01
    msg = 'Running Test Five: zscore p-value is "0.01".'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': '0.01'})
    ts.append(['t_6_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: zscore p-value is int of 1 (not supported)
    msg = 'Running Test Five: zscore p-value is "1" (not supported).'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': '1'})
    ts.append(['t_6_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     
            
    return ts

### Test Six: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_six_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Seven: IVT Standardisation

In [ ]:
def build_test_seven_funcs(in_nc, temp_nc):
    """sets up test seven functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': None,                  # upper quantile for standardisation
        'in_stand_qlower': None,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default ivt lower and upper
    msg = 'Running Test Seven: default IVT lower, upper input (0.05, 0.99).'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.05, 'in_stand_qupper': 0.99})
    ts.append(['t_7_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: ivt lower and upper "", ""
    msg = 'Running Test Seven: IVT lower, upper input "", "".'
    params = inputs.copy()
    params.update({'in_stand_qlower': '', 'in_stand_qupper': ''})
    ts.append(['t_7_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: ivt lower and upper 0.0, 0.99
    msg = 'Running Test Seven: IVT lower, upper input 0.0, 0.99.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.0, 'in_stand_qupper': 0.99})
    ts.append(['t_7_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: ivt lower and upper 0.05, 0.0
    msg = 'Running Test Seven: IVT lower, upper input 0.05, 0.0.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.05, 'in_stand_qupper': 0.0})
    ts.append(['t_7_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: ivt lower and upper 0.25, 0.80
    msg = 'Running Test Seven: IVT lower, upper input 0.25, 0.80.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.25, 'in_stand_qupper': 0.80})
    ts.append(['t_7_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: ivt lower and upper 0.4, 0.6
    msg = 'Running Test Seven: IVT lower, upper input 0.4, 0.6.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.4, 'in_stand_qupper': 0.6})
    ts.append(['t_7_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: ivt lower and upper 0.5, 0.5
    msg = 'Running Test Seven: IVT lower, upper input 0.5, 0.5.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.5, 'in_stand_qupper': 0.5})
    ts.append(['t_7_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     

    # func: ivt lower and upper "0.2", "0.8"
    msg = 'Running Test Seven: IVT lower, upper input "0.2", "0.8".'
    params = inputs.copy()
    params.update({'in_stand_qlower': "0.2", 'in_stand_qupper': "0.8"})
    ts.append(['t_7_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    return ts

### Test Seven: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_seven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Eight: Pixel Flags

In [ ]:
def build_test_eight_funcs(in_nc, temp_nc):
    """sets up test eight functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': '',                     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default pixel flags
    msg = 'Running Test Eight: pixel flags Valid;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid;Snow;Water'})
    ts.append(['t_8_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: pixel flags is ''
    msg = 'Running Test Eight: pixel flags is "".'
    params = inputs.copy()
    params.update({'in_fmask_flags': ''})
    ts.append(['t_8_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water
    msg = 'Running Test Eight: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'NoData;Valid;Cloud;Shadow;Snow;Water'})
    ts.append(['t_8_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: pixel flags is Valid
    msg = 'Running Test Eight: pixel flags is Valid.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid'})
    ts.append(['t_8_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is Cloud;Shadow
    msg = 'Running Test Eight: pixel flags is Cloud;Shadow.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Cloud;Shadow'})
    ts.append(['t_8_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is Water
    msg = 'Running Test Eight: pixel flags is Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Water'})
    ts.append(['t_8_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is Water;Water
    msg = 'Running Test Eight: pixel flags is Water;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Water;Water'})
    ts.append(['t_8_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is water
    msg = 'Running Test Eight: pixel flags is water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'water'})
    ts.append(['t_8_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    return ts

### Test Eight: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eight_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Nine: Max Cloud Cover

In [ ]:
def build_test_nine_funcs(in_nc, temp_nc):
    """sets up test nine functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': '',                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default max cloud cover
    msg = 'Running Test Nine: default max cloud (10).'
    params = inputs.copy()
    params.update({'in_max_cloud': 10})
    ts.append(['t_9_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: max cloud cover is 0
    msg = 'Running Test Nine: max cloud cover is 0.'
    params = inputs.copy()
    params.update({'in_max_cloud': 0})
    ts.append(['t_9_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: max cloud cover is 50
    msg = 'Running Test Nine: max cloud cover is 50.'
    params = inputs.copy()
    params.update({'in_max_cloud': 50})
    ts.append(['t_9_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: max cloud cover is "10"
    msg = 'Running Test Nine: max cloud cover is "10".'
    params = inputs.copy()
    params.update({'in_max_cloud': "10"})
    ts.append(['t_9_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])  
    
    # func: max cloud cover is "10"
    msg = 'Running Test Nine: max cloud cover is "10".'
    params = inputs.copy()
    params.update({'in_max_cloud': "10"})
    ts.append(['t_9_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])      

    # func: max cloud cover is 150
    msg = 'Running Test Nine: max cloud cover is 150.'
    params = inputs.copy()
    params.update({'in_max_cloud': 150})
    ts.append(['t_9_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])   
    
    # func: max cloud cover is ''
    msg = 'Running Test Nine: max cloud cover is "".'
    params = inputs.copy()
    params.update({'in_max_cloud': ''})
    ts.append(['t_9_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])   

    # func: max cloud cover is None
    msg = 'Running Test Nine: max cloud cover is None.'
    params = inputs.copy()
    params.update({'in_max_cloud': None})
    ts.append(['t_9_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])  
    
    return ts

### Test Nine: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_nine_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Ten: Interpolate

In [ ]:
def build_test_ten_funcs(in_nc, temp_nc):
    """sets up test ten functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': '',                     # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default interpolate (True)
    msg = 'Running Test Ten: default interpolate (True).'
    params = inputs.copy()
    params.update({'in_interpolate': True})
    ts.append(['t_10_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: interpolate is False
    msg = 'Running Test Ten: interpolate is False.'
    params = inputs.copy()
    params.update({'in_interpolate': False})
    ts.append(['t_10_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: interpolate is None
    msg = 'Running Test Ten: interpolate is None (default to True).'
    params = inputs.copy()
    params.update({'in_interpolate': None})
    ts.append(['t_10_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])       
            
    return ts

### Test Ten: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_ten_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Eleven: Add To Map

In [ ]:
def build_test_eleven_funcs(in_nc, temp_nc):
    """sets up test eleven functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': '',             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default add result to map (True)
    msg = 'Running Test Eleven: default add result to map (True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': True})
    ts.append(['t_11_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: add result to map is False
    msg = 'Running Test Eleven: add result to map is False.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': False})
    ts.append(['t_11_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: add result to map is None
    msg = 'Running Test Eleven: add result to map is None (default to True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': None})
    ts.append(['t_11_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])       
            
    return ts

### Test Eleven: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eleven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

In [ ]:
def execute(self, parameters, messages):
    """
    Executes the GDV Spectra Likelihood module.
    """

    # safe imports
    import os, sys        # arcgis comes with these
    import datetime       # arcgis comes with these
    import numpy as np    # arcgis comes with these

    # risky imports (not native to arcgis)
    try:
        import xarray as xr
        import dask
    except Exception as e:
        arcpy.AddError('Python libraries xarray and dask not installed.')
        arcpy.AddMessage(str(e))
        return

    # import tools
    try:
        # shared folder
        sys.path.append(FOLDER_SHARED)
        import arc, satfetcher, tools

        # module folder
        sys.path.append(FOLDER_MODULES)
        import gdvspectra, cog 
    except Exception as e:
        arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
        arcpy.AddMessage(str(e))
        return

    # disable future warnings
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=RuntimeWarning)
    warnings.simplefilter(action='ignore', category=dask.array.core.PerformanceWarning)

    # grab parameter values 
    in_nc = parameters[0].valueAsText            # raw input satellite netcdf
    out_nc = parameters[1].valueAsText           # output gdv likelihood netcdf
    in_wet_months = parameters[2].valueAsText    # wet months 
    in_dry_months = parameters[3].valueAsText    # dry months 
    in_veg_idx = parameters[4].value             # vege index name
    in_mst_idx = parameters[5].value             # moisture index name       
    in_aggregate = parameters[6].value           # aggregate output
    in_zscore_pvalue = parameters[7].value       # zscore pvalue
    in_ivt_qupper = parameters[8].value          # upper quantile for standardisation
    in_ivt_qlower = parameters[9].value          # lower quantile for standardisation
    in_fmask_flags = parameters[10].valueAsText  # fmask flag values
    in_max_cloud = parameters[11].value          # max cloud percentage
    in_interpolate = parameters[12].value        # interpolate missing pixels
    in_add_result_to_map = parameters[13].value  # add result to map



    # # # # #
    # notify user and set up progress bar
    arcpy.AddMessage('Beginning GDVSpectra Likelihood.')
    arcpy.SetProgressor(type='step', 
                        message='Preparing parameters...',
                        min_range=0, max_range=19)



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Loading and checking netcdf...')
    arcpy.SetProgressorPosition(1)

    try:
        # do quick lazy load of netcdf for checking
        ds = xr.open_dataset(in_nc)
    except Exception as e:
        arcpy.AddError('Could not quick load input satellite NetCDF data.')
        arcpy.AddMessage(str(e))
        return

    # check xr type, vars, coords, dims, attrs
    if not isinstance(ds, xr.Dataset):
        arcpy.AddError('Input NetCDF must be a xr dataset.')
        return
    elif len(ds) == 0:
        arcpy.AddError('Input NetCDF has no data/variables/bands.')
        return
    elif 'x' not in ds.dims or 'y' not in ds.dims or 'time' not in ds.dims:
        arcpy.AddError('Input NetCDF must have x, y and time dimensions.')
        return
    elif 'x' not in ds.coords or 'y' not in ds.coords or 'time' not in ds.coords:
        arcpy.AddError('Input NetCDF must have x, y and time coords.')
        return
    elif 'spatial_ref' not in ds.coords:
        arcpy.AddError('Input NetCDF must have a spatial_ref coord.')
        return
    elif len(ds['x']) == 0 or len(ds['y']) == 0 or len(ds['time']) == 0:
        arcpy.AddError('Input NetCDF must have all at least one x, y and time index.')
        return
    elif 'oa_fmask' not in ds and 'fmask' not in ds:
        arcpy.AddError('Expected cloud mask band not found in NetCDF.')
        return
    elif not hasattr(ds, 'time.year') or not hasattr(ds, 'time.month'):
        arcpy.AddError('Input NetCDF must have time with year and month component.')
        return
    elif len(ds.groupby('time.year')) < 3:
        arcpy.AddError('Input NetCDF must have >= 3 years of data.')
        return
    elif ds.attrs == {}:
        arcpy.AddError('NetCDF must have attributes.')
        return
    elif not hasattr(ds, 'crs'):
        arcpy.AddError('NetCDF CRS attribute not found. CRS required.')
        return
    elif ds.crs != 'EPSG:3577':
        arcpy.AddError('NetCDF CRS is not in GDA94 Albers (EPSG:3577).')            
        return 
    elif not hasattr(ds, 'nodatavals'):
        arcpy.AddError('NetCDF nodatavals attribute not found.')            
        return 

    # efficient: if all nan, 0 at first var, assume rest same, so abort
    if ds[list(ds)[0]].isnull().all() or (ds[list(ds)[0]] == 0).all():
        arcpy.AddError('NetCDF has empty variables. Please download again.')            
        return 

    try:
        # now, do proper open of netcdf properly (and set nodata to nan)
        ds = satfetcher.load_local_nc(nc_path=in_nc, 
                                      use_dask=True, 
                                      conform_nodata_to=np.nan)
    except Exception as e:
        arcpy.AddError('Could not properly load input satellite NetCDF data.')
        arcpy.AddMessage(str(e))
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Getting NetCDF attributes...')
    arcpy.SetProgressorPosition(2)

    # get attributes from dataset
    ds_attrs = ds.attrs
    ds_band_attrs = ds[list(ds)[0]].attrs
    ds_spatial_ref_attrs = ds['spatial_ref'].attrs

    # remove potential datetime duplicates
    ds = satfetcher.group_dupe_times(ds)



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Removing invalid pixels and empty dates...')
    arcpy.SetProgressorPosition(3)  

    # convert fmask as text to numeric code equivalents
    in_fmask_flags = [e for e in in_fmask_flags.split(';')]
    in_fmask_flags = arc.convert_fmask_codes(in_fmask_flags)

    # check if flags selected, if not, select all 
    if len(in_fmask_flags) == 0:
        arcpy.AddWarning('No flags selected, using default.')
        in_fmask_flags = [1, 4, 5]

    # check numeric flags are valid 
    for flag in in_fmask_flags:
        if flag not in [0, 1, 2, 3, 4, 5]:
            arcpy.AddError('Pixel flag not supported.')
            return

    # check for duplicate flags 
    u, c = np.unique(in_fmask_flags, return_counts=True)
    if len(u[c > 1]) > 0:
        arcpy.AddError('Duplicate pixel flags detected.')
        return

    # get name of mask band
    mask_band = arc.get_name_of_mask_band(list(ds))

    try:
        # remove invalid pixels and empty scenes
        ds = cog.remove_fmask_dates(ds=ds, 
                                    valid_class=in_fmask_flags, 
                                    max_invalid=in_max_cloud, 
                                    mask_band=mask_band, 
                                    nodata_value=np.nan, 
                                    drop_fmask=True)
    except Exception as e:
        arcpy.AddError('Could not mask pixels.')
        arcpy.AddMessage(str(e))
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Conforming satellite band names...')
    arcpy.SetProgressorPosition(4)

    try:
        # get platform name from attributes, error if no attributes
        in_platform = arc.get_platform_from_dea_attrs(ds_attrs)

        # conform dea aws band names based on platform
        ds = satfetcher.conform_dea_ard_band_names(ds=ds, 
                                                   platform=in_platform.lower())   
    except Exception as e: 
        arcpy.AddError('Could not get platform from attributes.')
        arcpy.AddMessage(str(e))
        return

    # check if all expected bands are in dataset 
    for band in ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']:
        if band not in ds:
            arcpy.AddError('NetCDF is missing band: {}. Need all bands.'.format(band))
            return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Reducing dataset to wet and dry months...')
    arcpy.SetProgressorPosition(5)   

    # prepare wet, dry season lists
    if in_wet_months == '' or in_dry_months == '':
        arcpy.AddError('Must include at least one wet and dry month.')
        return

    # unpack months
    wet_month = [int(e) for e in in_wet_months.split(';')]
    dry_month = [int(e) for e in in_dry_months.split(';')]

    # check if same months in wet and dry
    for v in wet_month:
        if v in dry_month:
            arcpy.AddError('Cannot use same month in wet and dry months.')
            return

    try:
        # reduce xr dataset into only wet, dry months (force rechunk via time)
        ds = gdvspectra.subset_months(ds=ds.chunk({'time': -1}), 
                                      month=wet_month + dry_month,
                                      inplace=True)
    except Exception as e: 
        arcpy.AddError('Could not subset dataset into wet and dry months.')
        arcpy.AddMessage(str(e))
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Calculating vegetation and moisture indices...')
    arcpy.SetProgressorPosition(6) 

    # check if veg idx supported 
    if in_veg_idx.lower() not in ['ndvi', 'evi', 'savi', 'msavi', 'slavi', 'mavi', 'kndvi', 'tcg']:
        arcpy.AddError('Vegetation index not supported.')
        return 
    elif in_mst_idx.lower() not in ['ndmi', 'gvmi']:
        arcpy.AddError('Moisture index not supported.')
        return 

    try:
        # calculate vegetation and moisture index
        ds = tools.calculate_indices(ds=ds, 
                                     index=[in_veg_idx.lower(), in_mst_idx.lower()], 
                                     custom_name=['veg_idx', 'mst_idx'], 
                                     rescale=True, 
                                     drop=True)

        # add band attrs back on
        ds['veg_idx'].attrs = ds_band_attrs   
        ds['mst_idx'].attrs = ds_band_attrs

    except Exception as e: 
        arcpy.AddError('Could not calculate indices.')
        arcpy.AddMessage(str(e))
        return

    # check once more: if all nan, 0, abort
    if ds['veg_idx'].isnull().all() or ds['mst_idx'].isnull().all():
        arcpy.AddError('NetCDF has empty variables. Please download again.')            
        return 



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Interpolating dataset, if requested...')
    arcpy.SetProgressorPosition(7) 

    # if requested...
    if in_interpolate:
        try:
            # interpolate along time dimension (linear)
            ds = tools.perform_interp(ds=ds, method='full')
        except Exception as e: 
            arcpy.AddError('Could not interpolate dataset.')
            arcpy.AddMessage(str(e))
            return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Computing data into memory, please wait...')
    arcpy.SetProgressorPosition(8)

    # compute! 
    ds = ds.compute()

    # check if all nan again
    if ds.to_array().isnull().all():
        arcpy.AddError('NetCDF is empty. Please download again.')            
        return 



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Resampling dataset to annual wet/dry medians...')
    arcpy.SetProgressorPosition(9)     

    # extract datetimes for wet and dry seasons 
    dts_wet = ds['time'].where(ds['time.month'].isin(wet_month), drop=True)
    dts_dry = ds['time'].where(ds['time.month'].isin(dry_month), drop=True)

    # check if wet/dry months exist in the dataset, arent all empty
    if len(dts_wet) == 0 or len(dts_dry) == 0:
        arcpy.AddError('No wet and/or dry months captured in NetCDF.')
        return
    elif ds.sel(time=dts_wet).to_array().isnull().all():
        arcpy.AddError('Entire wet season is devoid of values in NetCDF.')
        return
    elif ds.sel(time=dts_dry).to_array().isnull().all():
        arcpy.AddError('Entire dry season is devoid of values in NetCDF.')
        return

    try:
        # resample data to annual seasons
        ds = gdvspectra.resample_to_wet_dry_medians(ds=ds, 
                                                    wet_month=wet_month, 
                                                    dry_month=dry_month,
                                                    inplace=True)
    except Exception as e: 
            arcpy.AddError('Could not resample annualised wet and dry seasons.')
            arcpy.AddMessage(str(e))
            return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Removing outliers, if requested...')
    arcpy.SetProgressorPosition(10)

    # prepare zscore selection
    if in_zscore_pvalue not in [0.01, 0.05, 0.1, None]:
        arcpy.AddWarning('Z-score value not supported. Setting to default.')
        in_zscore_pvalue = None

    # if requested...
    if in_zscore_pvalue is not None:
        try:
            # remove outliers
            ds = gdvspectra.nullify_wet_dry_outliers(ds=ds, 
                                                     wet_month=wet_month, 
                                                     dry_month=dry_month, 
                                                     p_value=in_zscore_pvalue,
                                                     inplace=True)     
        except Exception as e: 
            arcpy.AddError('Could not remove outliers.')
            arcpy.AddMessage(str(e))
            return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Cleaning years with insufficient seasonality...')
    arcpy.SetProgressorPosition(11) 

    try:
        # remove any years missing wet, dry season 
        ds = gdvspectra.drop_incomplete_wet_dry_years(ds=ds)
    except Exception as e: 
        arcpy.AddError('Could not drop years with insufficient seasons.')
        arcpy.AddMessage(str(e))
        return

    # check if we still have sufficient number of years 
    if len(ds.groupby('time.year')) < 3:
        arcpy.AddError('Input NetCDF needs more years. Expand time range in NetCDF.')
        return

    try:
        # fill any empty first, last years using manual back/forward fill
        ds = gdvspectra.fill_empty_wet_dry_edges(ds=ds,
                                                 wet_month=wet_month, 
                                                 dry_month=dry_month,
                                                 inplace=True)
    except Exception as e: 
        arcpy.AddError('Could not fill empty wet and dry edge dates.')
        arcpy.AddMessage(str(e))
        return

    try:
        # interpolate missing values 
        ds = gdvspectra.interp_empty_wet_dry(ds=ds,
                                             wet_month=wet_month,
                                             dry_month=dry_month,
                                             method='full',
                                             inplace=True)
    except Exception as e: 
        arcpy.AddError('Could not interpolate empty wet and dry edge dates.')
        arcpy.AddMessage(str(e))
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Standardising data to dry season invariant targets...')
    arcpy.SetProgressorPosition(12)

    # check upper quantile
    if in_ivt_qlower < 0 or in_ivt_qlower >= 0.5:
        arcpy.AddMessage('Lower quantile must be between 0, 0.5. Setting to default.')
        in_ivt_qlower = 0.05

    # do same for upper quantile
    if in_ivt_qupper <= 0.5 or in_ivt_qupper > 1.0:
        arcpy.AddMessage('Upper quantile must be between 0.5, 1.0. Setting to default.')
        in_ivt_qlower = 0.99 

    # check if upper <= lower 
    if in_ivt_qupper <= in_ivt_qlower:
        arcpy.AddError('Upper quantile must be > than lower quantile value.')
        return

    try:
        # standardise data to invariant targets derived from dry times
        ds = gdvspectra.standardise_to_dry_targets(ds=ds, 
                                                   dry_month=dry_month, 
                                                   q_upper=in_ivt_qupper,
                                                   q_lower=in_ivt_qlower,
                                                   inplace=True)
    except Exception as e: 
        arcpy.AddError('Could not standardise data to invariant targets.')
        arcpy.AddMessage(str(e))
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Calculating seasonal similarity...')
    arcpy.SetProgressorPosition(13)  

    try:
        # calculate seasonal similarity
        ds_similarity = gdvspectra.calc_seasonal_similarity(ds=ds,
                                                            wet_month=wet_month,
                                                            dry_month=dry_month,
                                                            q_mask=0.9,
                                                            inplace=True)
    except Exception as e: 
        arcpy.AddError('Could not generate similarity.')
        arcpy.AddMessage(str(e))
        return

    # check similarity dataset is not empty 
    if ds_similarity.to_array().isnull().all():
        arcpy.AddError('Similarity modelling returned no data.')
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Calculating GDV Likelihood...')
    arcpy.SetProgressorPosition(14)  

    try:
        # calculate gdv likelihood
        ds = gdvspectra.calc_likelihood(ds=ds, 
                                        ds_similarity=ds_similarity,
                                        wet_month=wet_month, 
                                        dry_month=dry_month)

        # convert dataset back to float32
        ds = ds.astype('float32')

    except Exception as e: 
        arcpy.AddError('Could not generate likelihood data.')
        arcpy.AddMessage(str(e))
        return

    # check likelihood dataset is not empty 
    if ds.to_array().isnull().all():
        arcpy.AddError('Likelihood modelling returned no data.')
        return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Aggreating dataset, if requested...')
    arcpy.SetProgressorPosition(15) 

    # if requested...
    if in_aggregate is True:
        try:
            # reducing full dataset down to one median image without time 
            ds = ds.median('time')
        except Exception as e: 
            arcpy.AddError('Could not aggregate dataset.')
            arcpy.AddMessage(str(e))
            return



    # # # # #
    # notify and increment progess bar
    arcpy.SetProgressorLabel('Appending attributes back on to dataset...')
    arcpy.SetProgressorPosition(16)

    # append attrbutes on to dataset and bands
    ds.attrs = ds_attrs
    ds['spatial_ref'].attrs = ds_spatial_ref_attrs
    for var in ds:
        ds[var].attrs = ds_band_attrs



    # # # # #
    # notify and increment progess bar
    arcpy.SetProgressorLabel('Exporting NetCDF file...')
    arcpy.SetProgressorPosition(17)   

    try:
        # export netcdf file
        tools.export_xr_as_nc(ds=ds, filename=out_nc)
    except Exception as e: 
            arcpy.AddError('Could not export dataset.')
            arcpy.AddMessage(str(e))
            return



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Adding output to map, if requested...')
    arcpy.SetProgressorPosition(18)

    # if requested...
    if in_add_result_to_map:
        try:
            # for current project, open current map
            aprx = arcpy.mp.ArcGISProject('CURRENT')
            m = aprx.activeMap

            # remove likelihood layer if already exists
            for layer in m.listLayers():
                if layer.name == 'likelihood.crf':
                    m.removeLayer(layer)

            # create output folder using datetime as name
            dt = datetime.datetime.now().strftime('%d%m%Y%H%M%S')
            out_folder = os.path.join(os.path.dirname(out_nc), 'likelihood' + '_' + dt)
            os.makedirs(out_folder)

            # disable visualise on map temporarily
            arcpy.env.addOutputsToMap = False

            # create crf filename and copy it
            out_file = os.path.join(out_folder, 'likelihood.crf')
            crf = arcpy.CopyRaster_management(in_raster=out_nc, 
                                              out_rasterdataset=out_file)

            # add to map                  
            m.addDataFromPath(crf)   

        except Exception as e:
            arcpy.AddWarning('Could not visualise output, aborting visualisation.')
            arcpy.AddMessage(str(e))
            pass

        try:
            # get symbology, update it
            layer = m.listLayers('likelihood.crf')[0]
            sym = layer.symbology

            # if layer has stretch coloriser, apply color
            if hasattr(sym, 'colorizer'):
                if sym.colorizer.type == 'RasterStretchColorizer':

                    # apply percent clip type
                    sym.colorizer.stretchType = 'PercentClip'
                    sym.colorizer.minPercent = 0.01
                    sym.colorizer.maxPercent = 0.99

                    # apply color map
                    cmap = aprx.listColorRamps('Bathymetric Scale')[0]
                    sym.colorizer.colorRamp = cmap

                    # apply other basic options
                    sym.colorizer.invertColorRamp = False
                    sym.colorizer.gamma = 1.0

                    # update symbology
                    layer.symbology = sym

        except Exception as e:
            arcpy.AddWarning('Could not colorise output, aborting colorisation.')
            arcpy.AddMessage(str(e))
            pass



    # # # # #
    # clean up variables
    arcpy.SetProgressorLabel('Finalising process...')
    arcpy.SetProgressorPosition(19)

    # close main dataset and del datasets
    ds.close()
    del ds 

    # close similarity dataset
    ds_similarity.close()
    del ds_similarity

    # notify user
    arcpy.AddMessage('Generated GDV Likelihood successfully.')

    return

In [ ]:
def execute(self, parameters, messages):       
        """
        Executes the GDV Spectra Threshold module.
        """
        
        # safe imports
        import os, sys                           # arcgis comes with these
        import datetime                          # arcgis comes with this
        import numpy as np                       # arcgis comes with this
        import pandas as pd                      # arcgis comes with this
        import arcpy                             # arcgis comes with this
        
        # risky imports (not native to arcgis)
        try:
            import xarray as xr
            import dask
        except Exception as e:
            arcpy.AddError('Python libraries xarray and dask not installed.')
            arcpy.AddMessage(str(e))
            return
                
        # import tools
        try:
            # shared folder
            sys.path.append(FOLDER_SHARED)
            import arc, satfetcher, tools
        
            # module folder
            sys.path.append(FOLDER_MODULES)
            import gdvspectra 
        except Exception as e:
            arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
            arcpy.AddMessage(str(e))
            return
            
        # disable future warnings
        import warnings
        warnings.simplefilter(action='ignore', category=FutureWarning)
        warnings.simplefilter(action='ignore', category=RuntimeWarning)
        warnings.simplefilter(action='ignore', category=dask.array.core.PerformanceWarning)
        
        # grab parameter values 
        in_nc = parameters[0].valueAsText                # likelihood netcdf
        out_nc = parameters[1].valueAsText               # output netcdf
        in_aggregate = parameters[2].value               # aggregate dates
        in_specific_years = parameters[3].valueAsText    # set specific year 
        in_type = parameters[4].value                    # threshold type
        in_std_dev = parameters[5].value                 # std dev threshold value 
        in_occurrence_feat = parameters[6]               # occurrence shp path 
        in_pa_column = parameters[7].value               # occurrence shp pres/abse col 
        in_remove_stray = parameters[8].value            # apply salt n pepper -- requires sa
        in_convert_binary = parameters[9].value          # convert thresh to binary 1, nan
        in_add_result_to_map = parameters[10].value      # add result to map


        
        # # # # #
        # notify user and set up progress bar
        arcpy.AddMessage('Beginning GDVSpectra Threshold.')
        arcpy.SetProgressor(type='step', 
                            message='Preparing parameters...',
                            min_range=0, max_range=12)



        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Loading and checking netcdf...')
        arcpy.SetProgressorPosition(1)
        
        try:
            # do quick lazy load of netcdf for checking
            ds = xr.open_dataset(in_nc)
        except Exception as e:
            arcpy.AddWarning('Could not quick load input likelihood NetCDF data.')
            arcpy.AddMessage(str(e))
            return

        # check xr type, vars, coords, dims, attrs
        if not isinstance(ds, xr.Dataset):
            arcpy.AddError('Input NetCDF must be a xr dataset.')
            return
        elif len(ds) == 0:
            arcpy.AddError('Input NetCDF has no data/variables/bands.')
            return
        elif 'x' not in ds.dims or 'y' not in ds.dims:
            arcpy.AddError('Input NetCDF must have x, y dimensions.')
            return        
        elif 'x' not in ds.coords or 'y' not in ds.coords:
            arcpy.AddError('Input NetCDF must have x, y coords.')
            return
        elif 'spatial_ref' not in ds.coords:
            arcpy.AddError('Input NetCDF must have a spatial_ref coord.')
            return
        elif len(ds['x']) == 0 or len(ds['y']) == 0:
            arcpy.AddError('Input NetCDF must have at least one x, y index.')
            return
        elif 'like' not in ds:
            arcpy.AddError('Input NetCDF must have a "like" variable. Run GDVSpectra Likelihood.')
            return
        elif 'time' in ds and (not hasattr(ds, 'time.year') or not hasattr(ds, 'time.month')):
            arcpy.AddError('Input NetCDF must have time with year and month component.')
            return
        elif 'time' in ds.dims and 'time' not in ds.coords:
            arcpy.AddError('Input NetCDF has time dimension but not coordinate.')
            return
        elif ds.attrs == {}:
            arcpy.AddError('NetCDF attributes not found. NetCDF must have attributes.')
            return
        elif not hasattr(ds, 'crs'):
            arcpy.AddError('NetCDF CRS attribute not found. CRS required.')
            return
        elif ds.crs != 'EPSG:3577':
            arcpy.AddError('NetCDF CRS is not EPSG:3577. EPSG:3577 required.')            
            return 
        elif not hasattr(ds, 'nodatavals'):
            arcpy.AddError('NetCDF nodatavals attribute not found.')            
            return 

        # check if variables (should only be like) are empty
        if ds['like'].isnull().all() or (ds['like'] == 0).all():
            arcpy.AddError('NetCDF "like" variable is empty. Please download again.')            
            return 
 
        try:
            # now, do proper open of netcdf (set nodata to nan)
            ds = satfetcher.load_local_nc(nc_path=in_nc, 
                                          use_dask=True, 
                                          conform_nodata_to=np.nan)
        except Exception as e:
            arcpy.AddError('Could not properly load input likelihood NetCDF data.')
            arcpy.AddMessage(str(e))
            return



        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Getting NetCDF attributes...')
        arcpy.SetProgressorPosition(2)
            
        # get attributes from dataset
        ds_attrs = ds.attrs
        ds_band_attrs = ds[list(ds)[0]].attrs
        ds_spatial_ref_attrs = ds['spatial_ref'].attrs
        
        # remove potential datetime duplicates, if time exists
        if 'time' in ds:
            ds = satfetcher.group_dupe_times(ds)



        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Reducing dataset based on time, if requested...')
        arcpy.SetProgressorPosition(3)
                
        # if time is in dataset...
        if 'time' in ds:
        
            # check aggregate and specified year(s) is valid
            if in_aggregate is None:
                arcpy.AddError('Did not specify aggregate parameter.')
                return
            elif in_aggregate is False and in_specific_years is None:
                arcpy.AddError('Did not provide a specific year.')
                return
                
            # if specific years set...
            if in_aggregate is False:
                in_specific_years = [int(e) for e in in_specific_years.split(';')]
               
            # aggregate depending on user choice 
            if in_aggregate is True:
                ds = ds.median('time')
            else:
                ds = ds.where(ds['time.year'].isin(in_specific_years), drop=True)
                ds = ds.median('time')



        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Computing data into memory, please wait...')
        arcpy.SetProgressorPosition(4)

        # compute! 
        ds = ds.compute()
                
        # check if all nan again
        if ds.to_array().isnull().all():
            arcpy.AddError('NetCDF is empty. Please download again.')            
            return 
            
            
            
        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Preparing occurrence points, if provided...')
        arcpy.SetProgressorPosition(5)

        # we need nodataval attr, so ensure it exists 
        ds.attrs = ds_attrs        
        
        # if requested...
        if in_type == 'Occurrence Points':
        
            # check if both shapefile and field provided 
            if in_occurrence_feat.value is None or in_pa_column is None:
                arcpy.AddError('No occurrence feature and/or field provided.')
                return

            try:
                # get path to feature instead of map layer 
                desc = arcpy.Describe(in_occurrence_feat)
                in_occurrence_feat = os.path.join(desc.path, desc.name)
            
                # read shapefile via arcpy, convert feat into dataframe of x, y, actual
                df_records = arc.read_shp_for_threshold(in_occurrence_feat=in_occurrence_feat, 
                                                        in_pa_column=in_pa_column)

                # intersect points with dataset and extract likelihood values
                df_records = tools.intersect_records_with_xr(ds=ds, 
                                                             df_records=df_records, 
                                                             extract=True, 
                                                             res_factor=3, 
                                                             if_nodata='any')    

                # rename column to predicted and check
                df_records = df_records.rename(columns={'like': 'predicted'})
                
                # check if any records intersected dataset 
                if len(df_records.index) == 0:
                    arcpy.AddError('No shapefile points intersect GDV likelihood dataset.')
                    return
                    
                # remove any records where vars contain nodata
                df_records = tools.remove_nodata_records(df_records, 
                                                         nodata_value=ds.nodatavals)
                                                         
                # check again if any records exist
                if len(df_records.index) == 0:
                    arcpy.AddError('No shapefile points remain after empty values removed.')
                    return
                    
            except Exception as e:
                arcpy.AddError('Could not read shapefile, see messages for details.')
                arcpy.AddMessage(str(e))
                return
                
            # check if some 1s and 0s exist 
            unq = df_records['actual'].unique()
            if not np.any(unq == 1) or not np.any(unq == 0):
                arcpy.AddError('Insufficient presence/absence points within NetCDF bounds.')
                return



        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Thresholding GDV Likelihood...')
        arcpy.SetProgressorPosition(6)      
        
        try:
            # perform thresholding using either shapefile points or std dev
            if in_type == 'Occurrence Points' and df_records is not None:
                ds = gdvspectra.threshold_likelihood(ds=ds,
                                                     df=df_records, 
                                                     res_factor=3, 
                                                     if_nodata='any')
            else:
                ds = gdvspectra.threshold_likelihood(ds=ds,
                                                     num_stdevs=in_std_dev, 
                                                     res_factor=3, 
                                                     if_nodata='any')
                                                     
            # rename var, convert to float32
            ds = ds.rename({'like': 'thresh'}).astype('float32')

        except Exception as e:
            arcpy.AddError('Could not threshold data.')
            arcpy.AddMessage(str(e))
            #print(str(e))
            return
            
        # check if any data was returned after threshold
        if ds.to_array().isnull().all():
            arcpy.AddError('Threshold returned no values, try modifying threshold.')
            return



        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Removing stray pixels, if requested...')
        arcpy.SetProgressorPosition(7) 
        
        # if requested...
        if in_remove_stray:
            try:
                # remove salt n pepper 
                ds = gdvspectra.remove_salt_pepper(ds, iterations=1)
            except Exception as e:
                arcpy.AddError('Could not remove stray pixels.')
                arcpy.AddMessage(str(e))
                return



        # # # # #
        # notify and increment progess bar
        arcpy.SetProgressorLabel('Binarising values, if requested...')
        arcpy.SetProgressorPosition(8)
        
        # if requested...
        if in_convert_binary:

            # set all threshold non-nan values to 1
            ds = ds.where(ds.isnull(), 1)



        # # # # #
        # notify and increment progess bar
        arcpy.SetProgressorLabel('Appending attributes back on to dataset...')
        arcpy.SetProgressorPosition(9)
        
        # append attrbutes on to dataset and bands
        ds.attrs = ds_attrs
        ds['spatial_ref'].attrs = ds_spatial_ref_attrs
        for var in ds:
            ds[var].attrs = ds_band_attrs



        # # # # #
        # notify and increment progess bar
        arcpy.SetProgressorLabel('Exporting NetCDF file...')
        arcpy.SetProgressorPosition(10)   

        try:
            # export netcdf file
            tools.export_xr_as_nc(ds=ds, filename=out_nc)
        except Exception as e:
            arcpy.AddError('Could not export dataset.')
            arcpy.AddMessage(str(e))
            return



        # # # # #
        # notify and increment progess bar
        arcpy.SetProgressorLabel('Adding output to map, if requested...')
        arcpy.SetProgressorPosition(11)
        
        # if requested...
        if in_add_result_to_map:
            try:
                # for current project, open current map
                aprx = arcpy.mp.ArcGISProject('CURRENT')
                m = aprx.activeMap
                
                # remove threshold layer if already exists 
                for layer in m.listLayers():
                    if layer.name == 'likelihood_threshold.crf':
                        m.removeLayer(layer)
                
                # create output folder using datetime as name
                dt = datetime.datetime.now().strftime('%d%m%Y%H%M%S')
                out_folder = os.path.join(os.path.dirname(out_nc), 'likelihood_threshold' + '_' + dt)
                os.makedirs(out_folder)
            
                # disable visualise on map temporarily
                arcpy.env.addOutputsToMap = False
                
                # create crf filename and copy it
                out_file = os.path.join(out_folder, 'likelihood_threshold.crf')
                crf = arcpy.CopyRaster_management(in_raster=out_nc, 
                                                  out_rasterdataset=out_file)
                                    
                # add to map                  
                m.addDataFromPath(crf)  

            except Exception as e:
                arcpy.AddWarning('Could not visualise output, aborting visualisation.')
                arcpy.AddMessage(str(e))
                pass
                
            try:           
                # get symbology, update it
                layer = m.listLayers('likelihood_threshold.crf')[0]
                sym = layer.symbology
                
                # if layer has stretch coloriser, apply color
                if hasattr(sym, 'colorizer'):
                    
                    # apply percent clip type
                    sym.colorizer.stretchType = 'PercentClip'
                    sym.colorizer.minPercent = 0.01
                    sym.colorizer.maxPercent = 0.99
                
                    # colorise deopending on binary or continious
                    if in_convert_binary is True:
                        cmap = aprx.listColorRamps('Yellow to Red')[0]
                    else:
                        cmap = aprx.listColorRamps('Bathymetric Scale')[0]
                    
                    # apply colormap
                    sym.colorizer.colorRamp = cmap

                    # apply other basic options
                    sym.colorizer.invertColorRamp = False
                    sym.colorizer.gamma = 1.0
                        
                    # update symbology
                    layer.symbology = sym  
       
            
            except Exception as e:
                arcpy.AddWarning('Could not colorise output, aborting colorisation.')
                arcpy.AddMessage(str(e))
                pass


        # # # # #
        # notify and increment progress bar
        arcpy.SetProgressorLabel('Finalising process...')
        arcpy.SetProgressorPosition(12)
        
        # close main dataset
        ds.close()
        del ds 
        
        # notify user
        arcpy.AddMessage('Generated GDV Threshold successfully.')  
                
        return